# Update 1 
**Names:** Eva, Barbara & Joyce

In this update we have created a vector representation of the training set, in which a count is given to the words in each statement according to which word from the vocabulary it is. This representation can possibly be used to decide which words/features in the data are the most important and should be given the most weight. However, we were not sure yet how to do that...


Then, we tried some logistic regression on the data to predict which category ("false","barely-true", "half-true", "mostly-true", "true", "pants-fire") a statement belongs to. In this logistic regression we did not use the above-mentioned matrix, because we don't know yet how to incorporate it. 

In [71]:
#make a list of the vocabulary: all the distinct words occuring in the train set 
import nltk
from nltk.corpus import wordnet as wn
import csv
import string

allwords = []
with open("train.tsv", encoding="utf8") as tsvfile:    #open training set
    lines = csv.reader(tsvfile, delimiter="\t")        #convert file to lines
    for line in lines:
        statement = line[2]                                #get statement from each line 
        lostrings = statement.split(" ")                   #convert string to list of strings
        new_lostrings = []
        for word in lostrings:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation & lemmatize
            new_lostrings.append(word)
        allwords.extend(new_lostrings)

vocab = []                          #initialize a list for all the distint words in the trainset
for word in allwords:
    if word in vocab:               #do not add word if word is already in vocabulary 
        continue 
    else:
        vocab.append(word)


print("Number of words in trainset:", len(allwords))
print("Number of distinct words in trainset:", len(vocab))
print(allwords[:25])
print(vocab[:25])

Number of words in trainset: 184014
Number of distinct words in trainset: 11916
['say', 'the', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'on', 'demand', 'when', 'did', 'the', 'decline', 'of', 'coal', 'start', 'it', 'started', 'when', 'natural', 'gas', 'took', 'off']
['say', 'the', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'on', 'demand', 'when', 'did', 'decline', 'of', 'coal', 'start', 'it', 'started', 'natural', 'gas', 'took', 'off', 'that', 'to']


In [72]:
#make a list containing all the statements and count number of statements 

statements = []

with open("train.tsv", encoding="utf8") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        aline = line[2]
        bline = aline.split(" ")
        cline = []
        for word in bline:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation & lemmatize
            cline.append(word)
        statements.append(cline)


countlines = len(statements)
print("Number of statements:", countlines)
print("First 3 statements:", statements[:3])

Number of statements: 10240
First 3 statements: [['say', 'the', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'on', 'demand'], ['when', 'did', 'the', 'decline', 'of', 'coal', 'start', 'it', 'started', 'when', 'natural', 'gas', 'took', 'off', 'that', 'started', 'to', 'begin', 'in', 'president', 'george', 'w', 'bush', 'administration'], ['hillary', 'clinton', 'agrees', 'with', 'john', 'mccain', 'by', 'voting', 'to', 'give', 'george', 'bush', 'the', 'benefit', 'of', 'the', 'doubt', 'on', 'iran']]


In [87]:
#this list is similar to the previous list but it contains the label (true/false) at beginning of eacht statement 
statements = []

with open("train.tsv", encoding="utf8") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        aline = line[2]
        label = line[1]
        bline = aline.split(" ")
        cline = []
        for word in bline:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation
            cline.append(word)
        cline.insert(0, label)
        statements.append(cline)


countlines = len(statements)
print("Number of statements:", countlines)
print("First 3 statements:", statements[:3])

Number of statements: 10240
First 3 statements: [['false', 'say', 'the', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'on', 'demand'], ['half-true', 'when', 'did', 'the', 'decline', 'of', 'coal', 'start', 'it', 'started', 'when', 'natural', 'gas', 'took', 'off', 'that', 'started', 'to', 'begin', 'in', 'president', 'george', 'w', 'bush', 'administration'], ['mostly-true', 'hillary', 'clinton', 'agrees', 'with', 'john', 'mccain', 'by', 'voting', 'to', 'give', 'george', 'bush', 'the', 'benefit', 'of', 'the', 'doubt', 'on', 'iran']]


In [90]:
import numpy as np

labels ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4, "pants-fire":5}

rows = countlines
columns = len(vocab)+1 #add one for the truth-value 
matrix = np.zeros((rows, columns))

In [92]:
# creating the afore-mentioned matrix representation
counts1 = 0
for statement in statements:
    labelcount = labels[statement[0]] 
    matrix[counts1, 0] = labelcount
    counts2 = 1
    for word in vocab:
        if word in statement:
            count = statement.count(word)
            matrix[counts1, counts2] = count
        counts2 += 1 
    counts1 += 1 

print(matrix[0])

[ 0.  1.  1. ...,  0.  0.  0.]


In [95]:
# Testing using a smaller matrix

labels ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4}
lst = ["words", "that", "can", "occur", "in", "a","statement"]
example_statements =[["false", "a", "statement", "with", "words", "a"], ["half-true", "this", "is", "another", "example", "statement"], 
                     ["mostly-true","and","that", "was", "the", "last", "one"]]

example_matrix = np.zeros((len(example_statements), len(lst)+1))

ex_counts1 = 0
for statement in example_statements:
    labelcount = labels[statement[0]] 
    example_matrix[ex_counts1, 0] = labelcount
    ex_counts2 = 1
    for word in lst:
        if word in statement:
            count = statement.count(word)
            example_matrix[ex_counts1, ex_counts2] = count
        ex_counts2 += 1 
    ex_counts1 += 1 
    

print(example_matrix)

[[ 0.  1.  0.  0.  0.  0.  2.  1.]
 [ 2.  0.  0.  0.  0.  0.  0.  1.]
 [ 3.  0.  1.  0.  0.  0.  0.  0.]]


In [98]:
# different labeling 

labels ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4, "pants-fire":5}

lst = ["words", "that", "can", "occur", "in", "a","statement"]
example_statements =[["false", "a", "statement", "with", "words", "a"], ["half-true", "this", "is", "another", "example", "statement"], 
                     ["mostly-true","and","that", "was", "the", "last", "one"]]

example_matrix = np.zeros((len(example_statements), len(lst)+6))

ex_counts1 = 0
for statement in example_statements:
    labelcount = labels[statement[0]] 
    example_matrix[ex_counts1, labelcount] = 1
    ex_counts2 = 6
    for word in lst:
        if word in statement:
            count = statement.count(word)
            example_matrix[ex_counts1, ex_counts2] = count
        ex_counts2 += 1 
    ex_counts1 += 1 
    

print(example_matrix)

[[ 1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  2.  1.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


### Comments
> In the previous 2 cells we have tested whether the matrix representation worked out. In the first cell you can see that it did, in the second we tried a different matrix representation. 

> Namely, in the second matrix, we represented the truth-value in a seperate column for each truth-value, with 0 if the statement doesn't have that truth value, and 1 if it does. We were thinking this might be better, because if you give the truth value a number, then 'pants-fire' might be more considered as a important feature, because it always counts as 5, whereas 'false' counts as 0... Not super sure about this, tough. 


## Logistic regression

Here, we do logistic regression on the data, similar to what we have done in lab 9: "ML". 
- We first created a dataframe from the data
- Then created a new column in the df ("class"), representing the score for the different truth value. 
- Then made a count vector / TF-IDF representation of the statements as X and the belonging scores as y. 
- We then performed logistic regression, and tested it a bit filling in our own statements. 

In [46]:
import pandas as pd
import os, codecs

# create df
df_liar = pd.read_csv("train.tsv", encoding="utf8", sep="\t", names=["id", "truth-value", 
                                                                     "text", "topic", "name", "job", 
                                                                     "state", "politics", "count1", "count2", 
                                                                     "count3", "count4", "count5", "context"])

In [47]:
df_liar.head(5)

,id,truth-value,text,topic,name,job,state,politics,count1,count2,count3,count4,count5,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [52]:
labels ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4, "pants-fire":5}

def classify(text):
    return labels[text]

In [53]:
# add the class of truth-values
df_liar["class"] = df_liar["truth-value"].apply(classify)

In [54]:
df_liar.head(5)

,id,truth-value,text,topic,name,job,state,politics,count1,count2,count3,count4,count5,context,class
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,2
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,3
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,2


In [103]:
# TF-IDF representation of the text
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(df_liar.text)
X_train.shape

(10240, 12196)

In [104]:
# subtract the classes
y_train = df_liar["class"].values

In [105]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X_train, y_train)

/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [106]:
# testing
test1 = "Joyce is becoming the new president of America"
test2 = "Barbara just hijacked a plane! OMG"
test3 = "Eva wants to build a wall between the Netherlands and Belgium!"

x = count_vect.transform([test1])
x1 = count_vect.transform([test2])
x2 = count_vect.transform([test3])
print("prediction test 1:", logreg.predict(x)[0])
print("probability of belonging to classes:", logreg.predict_proba(x)[0])

print("prediction test 2:", logreg.predict(x1)[0])
print("probability of belonging to classes:", logreg.predict_proba(x1)[0])

print("prediction test 3:", logreg.predict(x2)[0])
print("probability of belonging to classes:", logreg.predict_proba(x2)[0])

prediction test 1: 0
probability of belonging to classes: [ 0.48589017  0.06160173  0.14523659  0.06214997  0.08505282  0.16006872]
prediction test 2: 3
probability of belonging to classes: [ 0.26817292  0.04392243  0.1608359   0.34184172  0.04104249  0.14418455]
prediction test 3: 2
probability of belonging to classes: [ 0.14191871  0.06550172  0.28323085  0.24729368  0.19961711  0.06243794]


### Evaluation
- We tried to evaluate the model, but because the pre-defined train and test set do not have the same vocabulary, they cannot be used to test... Or maybe in a way that we don't know yet. Below we have provided a test by just splitting up the training X into a train and test set, fixing this problem, but not really. 

In [108]:
df_liar_test = pd.read_csv("test.tsv", encoding="utf8", sep="\t", names=["id", "truth-value", 
                                                                     "text", "topic", "name", "job", 
                                                                     "state", "politics", "count1", "count2", 
                                                                     "count3", "count4", "count5", "context"])
df_liar_test["class"] = df_liar_test["truth-value"].apply(classify)

X_test = count_vect.fit_transform(df_liar_test.text)
y_test = df_liar_test["class"].values
X_test.shape

(1267, 4348)

In [109]:
# we could try to evaluate the model
logreg.fit(X_train, y_train)
y_hat_test = logreg.predict(X_test)

# evaluate using accuracy: proportion of correctly predicted over total
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_hat_test))
print(accuracy_score(y_test, y_hat_test, normalize=False))

/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model

ValueError: X has 4348 features per sample; expecting 12196

In [110]:
# splitting the matrices we used for training. 
from sklearn.model_selection import train_test_split

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.3, random_state=1)
X_train2, X_val, y_train2, y_val = train_test_split(X_train2, y_train2, test_size=0.5, random_state=1)

In [112]:
# evaluating again
logreg.fit(X_train2, y_train2)
y_hat_test2 = logreg.predict(X_test2)

# evaluate using accuracy: proportion of correctly predicted over total
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test2, y_hat_test2))
print(accuracy_score(y_test2, y_hat_test2, normalize=False))

/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.21875
672


In [ ]:
#multinomial logistic regression
import numpy as np

Xtrain = np.array([[1,    20,   30,   1],
                   [2,    22,   12,   33],
                   [3,    45,   65,   77],
                   [12,   43,   55,   65],
                   [11,   25,   30,   1],
                   [22,   23,   19,   31],
                   [31,   41,   11,   70],
                   [1,    48,   23,   60]])

ytrain = np.array([1, 2, 3, 4, 1, 2, 3, 4])
lr = LogisticRegression(solver='lbfgs',multi_class='multinomial').fit(Xtrain, ytrain)
yhat = lr.predict(Xtrain)
print(yhat)

#### Conclusion
This model is probably too small to provide accurate results, since the training set is now downgraded a lot... Therefore it has super low accuracy. 